In [1]:
#%pip install equinox openmdao[all]

In [2]:
import equinox as eqx
from util import generate_base_model, density, dyn_viscousity
import jax
import jax.numpy as jnp

In [3]:
base_model = generate_base_model()

lift_model = eqx.tree_deserialise_leaves(f"models/lift_surrogate.eqx", base_model)
drag_model = eqx.tree_deserialise_leaves(f"models/drag_surrogate.eqx", base_model)

In [4]:
from optimizer import Optimizer

In [5]:
opt = Optimizer()

In [6]:
opt.load_surrogates(lift_model, drag_model)

In [7]:
opt.define_material(
    E=69e9,
    rho=2700,
    yield_strength=241e6,
    shear_strength=207e6
)

In [8]:
altitude = 10_000

results = opt.solve_wing(
    lift_goal=4e5,
    safety_factor=0.8,
    v_infty=300,
    rho=density(altitude),
    mu=dyn_viscousity(altitude),
    alpha_geo=jnp.deg2rad(jnp.float32(5))
)

Optimizing planform
Optimization terminated successfully    (Exit mode 0)
            Current function value: 11056.86066360363
            Iterations: 5
            Function evaluations: 6
            Gradient evaluations: 5
Optimization Complete
-----------------------------------
Optimizing airfoil
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0056590866948928965
            Iterations: 13
            Function evaluations: 17
            Gradient evaluations: 13
Optimization Complete
-----------------------------------
Optimizing spars


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:441: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.9498775282617667
            Iterations: 16
            Function evaluations: 29
            Gradient evaluations: 15
Optimization Complete
-----------------------------------


In [9]:
from util import pretty_print

pretty_print(results)

Airfoil
----------
Base Shape Coef. 1.772
Max Thickness    0.191
Taper Exponent   2.940
Max Camber       0.014
Camber Exponent  0.992
Reflex           0.016

Planform
----------
Wingspan         24.565 m
Chord length     1.638 m
Aspect Ratio     15.000

Spars
----------
Flange Height    0.044 m
Flange Width     0.409 m
Web Height       0.293 m
Web Width        0.008 m
Spar Ratio       11.65
Main Spar Pos.   0.33c
Rear Spar Pos.   0.95c

Aerodynamics & Structure
----------
Lift             401140
Drag             9817
Cl at alpha=0    0.15
Alpha Cl=0       -1.36 deg
Max τ threshold  2.53%
Max σ threshold  80.00%
Material Usage   0.950 m3

Timing
----------
Planform         23 seconds
Airfoil          6 seconds
Spars            3 seconds
Total            34 seconds
